In [109]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt

#Ticker imported from wikipedia
SP500url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"

#Since impossible to get data from wikipedia I used a user agent
headers = {
    "User-Agent": ("Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                   "AppleWebKit/537.36 (KHTML, like Gecko) "
                   "Chrome/124.0.0.0 Safari/537.36")
}

tables = pd.read_html(SP500url, storage_options=headers)
SP500 = tables[0].rename(columns={"Symbol" : "Ticker", "Security" : "Name", "GICS Sector" : "Sector",	
                                  "GICS Sub-Industry" : "Sub-Sector"})
SP500 = SP500.set_index("Ticker")
#Test
SP500.loc["AAPL"]

Name                                                     Apple Inc.
Sector                                       Information Technology
Sub-Sector               Technology Hardware, Storage & Peripherals
Headquarters Location                         Cupertino, California
Date added                                               1982-11-30
CIK                                                          320193
Founded                                                        1977
Name: AAPL, dtype: object

In [111]:
def get_fin_info(ticker = str, period = "1y", trading_days = 252):
    ticker = yf.Ticker(ticker)
    # Historique de prix pour calcul de volatilité
    hist = t.history(period=period, auto_adjust=True)
    vol = None
    if not hist.empty:
        returns = np.log(hist["Close"]).diff().dropna()
        vol = returns.std(ddof=1) * np.sqrt(trading_days)
    
    # Fast info + info détaillée
    fast = getattr(t, "fast_info", {})
    try:
        info = t.info
    except Exception:
        info = {}
    return {
        "ticker": ticker,
        "name": info.get("longName", ticker),
        "currency": fast.get("currency") or info.get("currency"),
        "price": fast.get("last_price") or info.get("currentPrice"),
        "market_cap": fast.get("market_cap") or info.get("marketCap"),
        "pe_trailing": info.get("trailingPE"),
        "pe_forward": info.get("forwardPE"),
        "beta": info.get("beta"),
        "sector": info.get("sector"),
        "industry": info.get("industry"),
        "shares_outstanding": info.get("sharesOutstanding"),
        "volatility_1y": vol,
    }

get_fin_info("AAPL", "2y")

{'ticker': yfinance.Ticker object <AAPL>,
 'name': 'Apple Inc.',
 'currency': 'USD',
 'price': 239.49,
 'market_cap': 3554127380480,
 'pe_trailing': 36.341427,
 'pe_forward': 28.819494,
 'beta': 1.109,
 'sector': 'Technology',
 'industry': 'Consumer Electronics',
 'shares_outstanding': 14840399872,
 'volatility_1y': 0.27577219299449063}